# Importamos librerias - Creamos el dataframe de pericias

In [2]:
import pandas as pd
import numpy as np
from pprint import pprint

df = pd.read_csv('pericias_medicas.csv', sep=',',  encoding='utf-8')
#convierte en string los datos de la columna text
df = df['text'].apply(str)
df1 = df.head(20)
df1

0      \n\nInforma a continuación: \n\nJ.T. 13 \nPER...
1      \n\nJUZG. 40 \n\nPERITO MEDICO PRESENTA INFOR...
2     Expediente Nº CNT 74803/2014 \n \nJUZGADO NACI...
3     JUZGADO NACIONAL DE TRABAJO 59       LAVALLE 1...
4      \n\n \n\nPERITO PRESENTA INFORME \n\n \n\n \n...
5                           JNT N°72 \n\nPERITO MEDI...
6     J. T.  Nº 20              Exp. 9231//2016     ...
7     PERITO MEDICO PRESENTA INFORME \n\nSr. Juez: \...
8     II.- HISTORIA CLINICA \n\nPERITO MEDICO PRESEN...
9     JNT nº 67, Sito en Calle Lavalle nº 1268, Ciud...
10    PERITO MEDICO: PRESENTA PERITACION MEDICA.- \n...
11     \n\nSR. JUEZ: \n\nJUZGADO NACIONAL DEL TRABAJ...
12    PRESENTA DICTAMEN PERICIAL \n\nSeñor Juez: \n\...
13    JUZG. NAC. DEL TRABAJO Nº 12 \n\n             ...
14    SE PRESENTA – HACE SABER: \n\nJuz T 51 \n\n \n...
15    PERITO MÉDICO PRESENTA INFORME PERICIAL\n\nSr....
16    PRESENTA INFORME PERICIAL MÉDICO. \n\nJ.N.T. 1...
17    PERITO MEDICO PRESENTA INFORME PERICIAL \n

## Limpieza de texto

In [4]:
# Limpieza de texto
# Primera etapa de la limpieza de texto
import re, string, unicodedata
'''
Se hace la eliminacion de texto entre corchetes, acentos, signos de puntuacion (excepto . y :), palabras con numeros.
Se eliminan los espacios de sobra
Se eliminan \r, \t, \v, \f, \a
'''
def limpiarTexto1(txt: str, bert=False, nums=False) -> str:
    """
    Elimina caracteres no deseados
    Params:
        **txt**:texto a ser limpiado de caracteres no desaeados
    """
    if nums:
        txt = re.sub(r'\d+', ' ', txt)
    if not bert:
        txt = txt.translate(str.maketrans(
            'áéíóúýàèìòùÁÉÍÓÚÀÈÌÒÙÝ', 'aeiouyaeiouAEIOUAEIOUY'))

    '''
    Eliminamos caracteres especiales: tabulador orizontal(\t), tabulador vertical(\v), 
    retorno de carro(\r), avance de pagina(\f), 
    caracter de retroceso: Marca el límite de una palabra(\b), 
    '''
    txt = txt.replace('\r', ' ').replace("\v", ' ').replace(
        "\t", ' ').replace("\f", ' ').replace("\a", ' ').replace("\b", ' ')
    txt = re.sub(' +', ' ', txt)
    txt = txt.strip()
    return txt
df1.apply(str)
df1 = df1.apply(limpiarTexto1)


In [137]:
# Segunda fase de limpieza
# Se eliminan todos los elementos que meten ruido al texto y que no fueron eliminados en la fase de limpieza 1.
import re

def limpiarTexto2(text):
    text = re.sub('^ ',' ',text)
    text = re.sub(r"[\([{})\]]",'',text)
    text = re.sub('\n +\n','\n',text)
    text = re.sub(' +\n\n','\n',text)
    text = re.sub('\n\n+','\n',text)
    text = re.sub(' \n','\n',text)
    text = re.sub('\d\n','',text)
    text = re.sub('\x0c','',text)
    text = re.sub('\u200b\n','',text)
    text = re.sub('\d{3,100}','',text)
    text = re.sub('\d+-\d+','',text)
    text = re.sub('\x0c','',text)
    text = re.sub('[nN]º|[nN][. ]º','',text)
    text = re.sub('[º<>/]','',text)
    text = re.sub('\d{3,100}','',text)
    text = re.sub('[a-zA-z-.]+@[a-zA-Z]+.com','',text)
    return text

dfLimpio = df1.apply(limpiarTexto2)
#dfLimpio[0]

In [135]:
# Tercera fase de limpieza
# Eliminamos las lineas que no son de utilidad para el analisis o que van a afectar los resultados del mismo.
# Ejemplo de linea eliminada: las lineas que comienzan con "Se encuentra contestada en....."
import re

def limpiarTexto3(text):
    text = re.sub('[a-z1-9.]+[).-] [s|S]e encuentra contestad[a|o] .+[. \n]','',text)
    text = re.sub('[0-9]+[. ]+[yY]a fue contestado.+[.\n]','',text)
    text = re.sub('[fF]oja [1-9].+\n', '', text)
    text = re.sub('[pP]regunta[ 0-9]+[)].+\n|[rR]espuesta[ 0-9]+[)].+\n','',text)#elimina oraciones comenzadas en preguta/respuesta.
    return text

dfLimpio = dfLimpio.apply(limpiarTexto3)
dfLimpio = dfLimpio.apply(limpiarTexto2)
#dfLimpio[5]

In [85]:
#QUISE GUARDAR EL DFLIMPIO SIN LAS PALABRAS CON LONGITUD MENOR A 2.
#texto_nuevo = " "
#for w in dfLimpio:
#    if len(w)>2:
#        texto_nuevo = texto_nuevo + " " + w
#print(texto_nuevo)

# Búsqueda de títulos

In [143]:
# Búsqueda de títulos
# Se crea un nuevo dataframe de titulos según distintos criterios

# busca titulos que comienzan con numeros romanos
def buscarTitulosRomanos(text):
    # Expresiones regulares para encontrar los numeros romanos
    tituloNroRomano =re.compile(r'(I{1,3}|IV|V|VI{1,3}|IX|X)[-.)][\w\s]+[:\n]')
    titulosRomanosEncontrados = []

    for m in tituloNroRomano.finditer(text):
        titulosRomanosEncontrados.append(m.group())

    return titulosRomanosEncontrados

# titulosConNroRomano: lista que guarda los titulos que comienzan con numeros romanos   
titulosConNroRomano=[]
for expediente in dfLimpio:
    titulosConNroRomano.append(buscarTitulosRomanos(expediente))

#pprint(titulosConNroRomano)

In [148]:
# Busca titulos que comienzan con numeros latinos
def buscarTitulosLatinos(text):
    # Expresion regular para encontrar titulos que comienzan con numeros
    tituloNroLatino =re.compile(r'[1-9][-.)][\w\s]+[:\n]')
    titulosLatinosEncontrados = []

    for m in tituloNroLatino.finditer(text):
        titulosLatinosEncontrados.append(m.group())

    return titulosLatinosEncontrados

# titulosConNroLatino: lista que guarda los titulos que comienzan con numeros latinos
titulosConNroLatino=[]
for expediente in dfLimpio:
    titulosConNroLatino.append(buscarTitulosLatinos(expediente))
pprint(titulosConNroLatino)

[['5-SEMIOLOGIA DE LAS AFECCIONES:',
  '2. TOBILLO:',
  '4. PSICOENTREVISTA:',
  '1.\n'
  'Las secuelas en el tobillo izquierdo que presenta el actor es '
  'consecuencia\n',
  '2.\n'
  'Estas conclusiones son en base al hecho que aun persiste las dificultades\n',
  '6.\n'
  'Las dolencias y dificultades para la movilidad que presenta el actor\n'
  'determina que presente dificultades intermedias para la realizacion de '
  'tareas\n',
  '7.\nQue a los efectos fiscales denuncio mi CUIT :',
  '1-De la parte actora\n',
  '2. Ya fue contestado en el informe pericial\n',
  '3. Ya fue contestado en el informe pericial\n',
  '5. Ya fue contestado en el informe pericial\n',
  '6. Ya fue contestado en el informe pericial\n',
  '2. Ya fue contestado en el informe pericial\n',
  '3. Ya fue contestado en el informe pericial\n',
  '1-De la parte demandada\n',
  '1. Ya fue contestado en el informe pericial\n',
  '2. Ya fue contestado en el presente informe\n',
  '3. Ya fue contestado en el informe p

In [141]:
# Busca titulos en mayusculas
def buscarTitulosMayusculas(text):
    # Expresion regular para encontrar titulos en mayusculas.
    tituloMayusculas =re.compile(r'([1-9]|I{1,3}|IV|V|VI{1,3}|IX|X)[-.)][A-Z -]+[:.\n]')
    titulosMayusculasEncontrados = []

    for m in tituloMayusculas.finditer(text):
        titulosMayusculasEncontrados.append(m.group())

    return titulosMayusculasEncontrados
# titulosMAyuscula: lista que guarda los titulos en mayusculas
titulosMayusculas=[]
for expediente in dfLimpio:
    titulosMayusculas.append(buscarTitulosMayusculas(expediente))
    #print(titulosMayusculas)


#### Eliminacion de stopwords y lematizacion.

In [125]:
import nltk
import re

stops = nltk.corpus.stopwords.words('spanish')

#elimino stopwords
def remove_stops(texto: str) -> str:
    """
    Función que elimina stopwords
    Params:
        **texto**:texto a ser limpiado de stopwords

    """
    texto = [
        i for i in texto.split() if i not in stops
    ]
    return texto

def limpiarTitulos(text):
    text = re.sub(r'(I{1,3}|IV|V|VI{1,3}|IX|X)[-.)]','',text)
    text = re.sub('\d+','',text)
    text = re.sub('[.-:]','',text)
    text = re.sub('[\W][A-Z][\W]','',text)
    text = re.sub('[,-@)]','',text)
   
    return text

titulosMayusculasStop = limpiarTitulos(str(titulosMayusculas))
titulosMayusculasStop = remove_stops(str(titulosMayusculasStop))
pprint(titulosMayusculasStop)

["[['PROEMIO'",
 "'ANTECEDENTES",
 'DE',
 'INTERES',
 "MEDICOLEGAL'",
 "'EXAMEN",
 'FISICO',
 'DEL',
 "ACTOR'",
 "'SEMIOLOGIA",
 'DE',
 'LAS',
 "AFECCIONES'",
 "'",
 "TOBILLO'",
 "'",
 "PSICOENTREVISTA'",
 "'CONSIDERACIONES",
 "MEDICOLEGALES'",
 "'CONCLUSIONES'",
 "''",
 "'INCAPACIDAD'",
 "'PETITORIO'",
 ']',
 '[',
 "'",
 "PREAMBULO'",
 "'",
 'ANTECEDENTES',
 'DE',
 'INTERES',
 'MEDICO',
 'PRESENTE',
 'EN',
 "AUTOS'",
 "'",
 'FECHALUGAR',
 'DE',
 "EVALUACION'",
 "'",
 'PROCESO',
 'QUE',
 'MOTIVA',
 'EL',
 "EXAMEN'",
 "'",
 'ANTECEDENTES',
 "FAMILIARES'",
 "'",
 'ANTECEDENTES',
 "PERSONALES'",
 "'",
 "SEMIOLOGIA\\n'",
 "'",
 "PETITORIO'",
 ']',
 '[',
 "n']",
 "['TELEFONO\\n'",
 ']',
 '[',
 ']',
 '[]',
 "[']",
 '[',
 ']',
 "['",
 'HISTORIA',
 "CLINICA\\n'",
 "'",
 'ANTECEDENTES',
 "PERSONALES\\n'",
 "'ANTECEDENTES",
 'DE',
 'INTERES',
 'MEDICO',
 "LEGAL\\n'",
 "'",
 "'",
 'EXAMEN',
 "FISICO'",
 "'",
 'EXAMEN',
 "PSIQUICO'",
 "'",
 'PUNTOS',
 "PERICIALES\\n'",
 "'",
 'ESTUDIOS',
 "SOLICIT

#### DataFrame de los títulos en mayúsculas.

In [159]:
import pandas as pd
dfTitulosMayusculas = pd.DataFrame(titulosMayusculasStop,columns=['Tokens'])
dfTitulosMayusculas


,Tokens
0,[['PROEMIO'
1,'ANTECEDENTES
2,DE
3,INTERES
4,MEDICOLEGAL'
...,...
177,]
178,[
179,'II\n'
180,n']


In [127]:
#Cantidad de veces que se repiten los tokens en los titulos.
import pandas as pd
FrecuenciaTitMay = pd.DataFrame(titulosMayusculasStop)
count = FrecuenciaTitMay.value_counts()
print(count)

'               36
]               12
[                9
DE               9
ANTECEDENTES     5
                ..
FISICO           1
FISICA           1
FECHALUGAR       1
FAMILIARES'      1
'\n'             1
Length: 93, dtype: int64


In [128]:
#def word_count(titulosMayusculasStop):
 #   import collections
  #  word_freq = collections.defaultdict(int)
   # with open(titulosMayusculasStop) as f:
    #    for l in f:
     #       for w in l.strip().split():  
      #          word_freq[w] += 1
    #return word_freq



In [104]:

#Pasé a dataframe lo anterior
import pandas as pd
dfFrecuenciaTitMay = pd.DataFrame(count)
dfFrecuenciaTitMay

,0
0,
',43
],12
DE,11
[,9
[',4
...,...
IMPOTENCIA,1
AFECCIONES',1
HISTORIA,1


In [160]:
#No hay valores vacios
dfFrecuenciaTitMay.isna().sum 

<bound method NDFrame._add_numeric_operations.<locals>.sum of                  0
0                 
'            False
]            False
DE           False
[            False
['           False
...            ...
IMPOTENCIA   False
AFECCIONES'  False
HISTORIA     False
HECHOS       False
LA           False

[102 rows x 1 columns]>

In [162]:
# Indicamos las características que presentará cada nube de palabras.
wc = WordCloud(stopwords = titulosMayusculasStop, background_color = "black",
               colormap = "hsv", max_font_size = 150, random_state = 123)

NameError: name 'WordCloud' is not defined

In [169]:
 #guia para hacer la nube de palabras
 import matplotlib.pyplot as plt
 from  wordcloud import WordCloud

# Generamos las nubes de palabras para cada uno de los posts.
for index, i in enumerate(post.columns):
     wc.generate(post.texto_tokenizado[i])
     plt.subplot(3, 2, index+1)
     plt.imshow(wc, interpolation = "bilinear")
     plt.axis("off")
     plt.savefig("imagenes/.png")

# Mostramos las nubes de palabras resultantes.
plt.show()

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
from wordcloud import WordCloud 
import matplotlib.pyplot as plt 
import csv 
  
dfFrecuenciaTitMay
  
reader_ob = csv.reader(dfFrecuenciaTitMay) 
  
reader_contents = list(reader_ob) 
  
text = "" 
  
for row in reader_contents : 
      
    
    for word in row : 
  
        
        text = text + " " + word 
  
wordcloud = WordCloud(width=480, height=480, max_font_size=20, min_font_size=10).generate(text) 
plt.figure() 
plt.imshow(wordcloud, interpolation="bilinear") 
plt.axis("off") 
plt.margins(x=0, y=0) 
plt.show() 

In [161]:
from wordcloud import WordCloud
wordcloud = WordCloud().generate(dfFrecuenciaTitMay)

ModuleNotFoundError: No module named 'wordcloud'

#### DataFrame de los títulos con números romanos.

In [138]:
titulosRomanosStop = limpiarTitulos(str(titulosConNroRomano))
titulosRomanosStop = remove_stops(str(titulosRomanosStop))
pprint(titulosRomanosStop)

["[['PROEMIO'",
 "'EXAMEN",
 'FISICO',
 'DEL',
 "ACTOR'",
 "'",
 "PROSOGRAFIA'",
 "'",
 'MOTIVO',
 'DE',
 'LA',
 "EVALUACION'",
 "'",
 'DATOS',
 "PERSONALES'",
 "'",
 'RELATO',
 'DE',
 'LOS',
 "HECHOS'",
 'nLa',
 'articulacion',
 'factible',
 'inestabilidad',
 'actor',
 'sufra',
 'lesion\\nEl',
 'actor',
 'deberia',
 'completar',
 'tratamiento',
 'kinesico',
 'fin',
 'fortificar',
 "los\\n'",
 "'INCAPACIDAD'",
 "'PETITORIO']",
 "['",
 'PRONOSTICOTRATAMIENTO',
 'REQUERIDO',
 'EN',
 'EL',
 "FUTURO']",
 "['",
 'Este',
 'baremo',
 'conocido',
 'el\\nnombre',
 'Baremo',
 'Academia',
 'Nacional',
 'Ciencias',
 'Buenos',
 'Aires',
 "tambien\\n'",
 "n']",
 '[]',
 "['",
 'Miño',
 'Esteban',
 "Ignacio\\n']",
 "['",
 'II',
 "ambos\\n'",
 'entregado',
 'Informe',
 'Pericial\\nProveer',
 "conformidad\\n']",
 '[]',
 '[]',
 '[y',
 'respetuosamente',
 "dice'",
 "autos\\n']",
 "['",
 'PROEMIO\\n',
 'SEÑOR',
 "JUEZ'",
 "'",
 'Datos',
 'interes',
 'medico',
 "legal\\nACTORA'",
 "'",
 'EVALUACION',
 "CLIN

In [158]:
import pandas as pd
dfTitulosRomanos = pd.DataFrame(titulosRomanosStop,columns=['Tokens'])
dfTitulosRomanos


,Tokens
0,[['PROEMIO'
1,'EXAMEN
2,FISICO
3,DEL
4,ACTOR'
...,...
142,PREEXISTENTE\n'
143,n
144,SERA
145,JUSTICIA\n']


In [140]:
#Cantidad de veces que se repiten los tokens en los titulos.
import pandas as pd
FrecuenciaTitRomanos = pd.DataFrame(titulosRomanosStop)
cantidadRom = FrecuenciaTitRomanos.value_counts()
print(cantidadRom)

'                 13
[]                10
['                 5
DE                 3
actor              2
                  ..
Informe            1
Ignacio\n']        1
INCAPACIDAD\n'     1
II                 1
veracidad          1
Length: 119, dtype: int64


#### DataFrame de los títulos con números latinos.

In [149]:
titulosLatinosStop = limpiarTitulos(str(titulosConNroLatino))
titulosLatinosStop = remove_stops(str(titulosLatinosStop))
pprint(titulosLatinosStop)

["[['SEMIOLOGIA",
 'DE',
 'LAS',
 "AFECCIONES'",
 "'",
 "TOBILLO'",
 "'",
 "PSICOENTREVISTA'",
 "'\\nLas",
 'secuelas',
 'tobillo',
 'izquierdo',
 'presenta',
 'actor',
 "consecuencia\\n'",
 "'\\nEstas",
 'conclusiones',
 'base',
 'hecho',
 'aun',
 'persiste',
 "dificultades\\n'",
 "'\\nLas",
 'dolencias',
 'dificultades',
 'movilidad',
 'presenta',
 'actor\\ndetermina',
 'presente',
 'dificultades',
 'intermedias',
 'realizacion',
 "tareas\\n'",
 "'\\nQue",
 'efectos',
 'fiscales',
 'denuncio',
 'CUIT',
 "'",
 "'De",
 'parte',
 "actora\\n'",
 "'",
 'Ya',
 'contestado',
 'informe',
 "pericial\\n'",
 "'",
 'Ya',
 'contestado',
 'informe',
 "pericial\\n'",
 "'",
 'Ya',
 'contestado',
 'informe',
 "pericial\\n'",
 "'",
 'Ya',
 'contestado',
 'informe',
 "pericial\\n'",
 "'",
 'Ya',
 'contestado',
 'informe',
 "pericial\\n'",
 "'",
 'Ya',
 'contestado',
 'informe',
 "pericial\\n'",
 "'De",
 'parte',
 "demandada\\n'",
 "'",
 'Ya',
 'contestado',
 'informe',
 "pericial\\n'",
 "'",
 'Ya',
 'c

In [157]:
import pandas as pd
dfTitulosLatinos = pd.DataFrame(titulosLatinosStop, columns=['Tokens'])

#from IPython.display import HTML, display_html, display
#display(HTML('<h2>Frecuencia de los titulos con números latinos</h2>'))
#display_html(dfTitulosLatinos)
dfTitulosLatinos



,Tokens
0,[['SEMIOLOGIA
1,DE
2,LAS
3,AFECCIONES'
4,'
...,...
461,estimar
462,en\n']
463,[]
464,[]


In [150]:
#Cantidad de veces que se repiten los tokens en los titulos.
import pandas as pd
FrecuenciaTitLatinos = pd.DataFrame(titulosLatinosStop)
cantidadLatinos = FrecuenciaTitLatinos.value_counts()
print(cantidadLatinos)

'              59
contestado     12
Ya             12
informe        11
pericial\n'    10
               ..
dominantes      1
efectos         1
efectue         1
Este            1
existe\n'       1
Length: 289, dtype: int64


In [152]:
#Queria saber cuales son los stopwords que hay 
#from nltk.corpus import stopwords
#stopwords.words('spanish')

In [101]:
from sklearn.feature_extraction.text import CountVectorizer 

documento = df1.apply(str)  
# crear la transformación  
vectorizer = CountVectorizer() 
# tokenizar y construir el vocabulario
vectorizer.fit(documento) 
  
print("Vocabulary: ", vectorizer.vocabulary_) 
# codificador de documentos
vector = vectorizer.transform(documento) 

# resumir vector codificado
#print("El documento codificado es:") 
#print(vector.shape)
#print(type(vector))
#print(vector.toarray())

Vocabulary:  {'informa': 3399, 'continuacion': 1609, '13': 56, 'perito': 4576, 'medico': 3979, 'presenta': 4775, 'informe': 3407, 'pericial': 4566, 'autos': 957, 'boiero': 1047, 'franco': 2930, 'ivan': 3570, 'swiss': 5793, 'medical': 3969, 'art': 859, 'accidente': 438, 'ley': 3732, 'especial': 2512, 'expediente': 2706, '71399': 341, '2017': 147, 'proemio': 4859, 'sr': 5689, 'juez': 3610, 'nacional': 4182, 'hortas': 3214, 'maria': 3916, 'andrea': 727, 'de': 1833, 'oficio': 4350, 'en': 2395, 'los': 3820, 'siguientes': 5585, 'con': 1444, 'domicilio': 2250, 'constituido': 1572, 'la': 3657, 'calle': 1119, 'austria': 944, '1754': 94, '3º': 244, 'piso': 4645, 'dpto': 2265, '10': 26, 'esta': 2548, 'capital': 1142, '172': 92, 'tel': 5843, '4325': 258, '8802': 375, 'electronico': 2348, 'mashortas': 3930, 'hotmail': 3218, 'com': 1364, '27': 186, '16523394': 87, 'cumplimiento': 1793, 'lo': 3798, 'dispuesto': 2208, 'por': 4694, 'referente': 5164, 'presentes': 4784, 'despues': 2057, 'haber': 3092, '

In [ ]:
# r'.\n +'[0-9]+[-. ]+[a-zA-z ]+'

#.\n +'([vV][.-]+ [A-Za-z ]+)|([iI{0,3}][vV]+[.-]+ [A-Za-z ]+)